In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [2]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [4]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [5]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK']='1'
os.environ['force_download']='True'

In [6]:
from sentence_transformers import SentenceTransformer

In [9]:
# torch.backends.mps.is_available()
# mps_device = torch.device("mps")

In [10]:
#mps_device

In [11]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

/Users/as/.pyenv/versions/3.8.10/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/as/.pyenv/versions/3.8.10/lib/python3.8/site-packages/transformers/models/mpnet/modeling_mpnet.py:1052: UserWarning: The operator 'aten::cumsum.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask


In [12]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import torch

# Example tensor
x = torch.tensor([1, 2, 3, 4, 5], device='mps')
# Use cumsum
result = torch.cumsum(x, dim=0)
print(result)


tensor([ 1,  3,  6, 10, 15], device='mps:0')


In [13]:
vectors.shape

(8, 768)

In [14]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [15]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [16]:
index.add(vectors)

In [17]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x31338b810> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [18]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [19]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [20]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [23]:
distances, I = index.search(svec, k=2)
distances

array([[1.4405564, 1.4696717]], dtype=float32)

In [24]:
I

array([[3, 2]])

In [25]:
I.tolist()

[[3, 2]]

In [26]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [27]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [28]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query